## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [10]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [12]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [16]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) #output_dim=100,activation=relu
classifier.add(BatchNormalization())
classifier.add(Dropout(rate=0.5))

#輸出
classifier.add(Dense(units=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 4s 9ms/step - loss: 2.1662 - accuracy: 0.2849
Epoch 2/100
500/500 [==============================] - 4s 9ms/step - loss: 1.7554 - accuracy: 0.3840
Epoch 3/100
500/500 [==============================] - 5s 9ms/step - loss: 1.6550 - accuracy: 0.4202
Epoch 4/100
500/500 [==============================] - 4s 9ms/step - loss: 1.5974 - accuracy: 0.4416
Epoch 5/100
500/500 [==============================] - 4s 9ms/step - loss: 1.5586 - accuracy: 0.4559
Epoch 6/100
500/500 [==============================] - 5s 9ms/step - loss: 1.5273 - accuracy: 0.4652
Epoch 7/100
500/500 [==============================] - 4s 9ms/step - loss: 1.4979 - accuracy: 0.4792
Epoch 8/100
500/500 [==============================] - 4s 8ms/step - loss: 1.4744 - accuracy: 0.4863
Epoch 9/100
500/500 [==============================] - 4s 9ms/step - loss: 1.4574 - accuracy: 0.4895
Epoch 10/100
500/500 [==============================] - 4s 9ms/step - loss: 1.4411 - accura

500/500 [==============================] - 4s 9ms/step - loss: 1.2533 - accuracy: 0.5670
Epoch 82/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2497 - accuracy: 0.5700
Epoch 83/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2516 - accuracy: 0.5686
Epoch 84/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2524 - accuracy: 0.5702
Epoch 85/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2503 - accuracy: 0.5700
Epoch 86/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2480 - accuracy: 0.5693
Epoch 87/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2500 - accuracy: 0.5688
Epoch 88/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2468 - accuracy: 0.5708
Epoch 89/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2449 - accuracy: 0.5683
Epoch 90/100
500/500 [==============================] - 4s 9ms/step - loss: 1.2486 - accuracy: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [17]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.01149131, 0.00195824, 0.06002292, 0.57561296, 0.10072201,
        0.18515892, 0.01455909, 0.03842798, 0.00698881, 0.00505778]],
      dtype=float32)